[View in Colaboratory](https://colab.research.google.com/github/pgurazada/Exploring-Deep-Learning/blob/master/2018_06_09_reusing_convnets.ipynb)

### Connect Google Drive and provide data access to the VM

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

In [0]:
import getpass

In [0]:
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls drive/data

### Model fit and analysis

In [0]:
from keras.applications import VGG16

In [0]:
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [0]:
conv_base.summary()

In [0]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_dir = 'drive/data/train'
validation_dir = 'drive/data/validation'

In [0]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 10

In [0]:
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    
    generator = datagen.flow_from_directory(directory,
                                            target_size=(150, 150),
                                            batch_size=batch_size,
                                            class_mode='binary')
    
    i = 0
    
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    
    return features, labels

In [0]:
train_features, train_labels = extract_features(train_dir, 2000)

In [0]:
validation_features, validation_labels = extract_features(validation_dir, 1000)

In [0]:
train_features.shape

In [0]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))

In [0]:
train_features.shape

In [0]:
from keras.models import Sequential

from keras.layers import Dense, Activation, Dropout, Flatten

from keras.losses import binary_crossentropy

from keras.optimizers import RMSprop

from keras.metrics import binary_accuracy

from keras.callbacks import EarlyStopping

from keras import backend as K

In [0]:
model = Sequential()

model.add(Dense(256, input_dim=4*4*512))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [0]:
model.summary()

In [0]:
model.compile(optimizer=RMSprop(lr=2e-5),
              loss=binary_crossentropy,
              metrics=[binary_accuracy])

In [0]:
model_output = model.fit(train_features, train_labels,
                         epochs=30,
                         batch_size=20,
                         validation_data=(validation_features, validation_labels),
                         callbacks=[EarlyStopping(patience=4)])

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set_style('ticks')
sns.set_context('talk', font_scale=1.2)

In [0]:
epochs = range(len(model_output.history['binary_accuracy']))


In [0]:
plt.figure(figsize=(12, 6))
plt.plot(epochs, model_output.history['binary_accuracy'], '--', label='Training accuracy')
plt.plot(epochs, model_output.history['val_binary_accuracy'], '-', label='Validation accuracy')
plt.title('Validation and training accuracy')
plt.legend()

In [0]:
plt.figure(figsize=(12, 6))
plt.plot(epochs, model_output.history['loss'], '--', label='Training loss')
plt.plot(epochs, model_output.history['val_loss'], '-', label='Validation loss')
plt.title('Validation and training loss')
plt.legend()

In [0]:
K.clear_session()

In [0]:
model = Sequential()

model.add(conv_base)

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [0]:
model.summary()

In [0]:
conv_base.trainable = False

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow_from_directory(# This is the target directory
                                                    train_dir,
                                                    # All images will be resized to 150x150
                                                    target_size=(150, 150),
                                                    batch_size=5,
                                                    # Since we use binary_crossentropy loss, we need binary labels
                                                    class_mode='binary')

In [0]:
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=5,
                                                        class_mode='binary')

In [0]:
model.compile(loss=binary_crossentropy,
              optimizer=RMSprop(lr=2e-5),
              metrics=[binary_accuracy])

In [0]:
model_output = model.fit_generator(train_generator,
                                   steps_per_epoch=400,
                                   epochs=30,
                                   validation_data=validation_generator,
                                   validation_steps=200,
                                   callbacks=[EarlyStopping(patience=4)])